<a href="https://colab.research.google.com/github/AmbrogioMB/AlgOpt/blob/main/Short_Term_Financing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. The problem
Corporations routinely face the problem of financing short term cash commitments.
Linear programming can help in figuring out an optimal combination
of financial instruments to meet these commitments.

A company has the following short term financing problem.

$$
\begin{array}{|c|r|}
\hline
\textbf{Month} & \textbf{Net cash flow} \\
\hline
\text{Jan} & -150 \\
\text{Feb} & -100 \\
\text{Mar} & 200 \\
\text{Apr} & -200 \\
\text{May} & 50 \\
\text{Jun} & 300 \\
\hline
\end{array}
$$

Net cash flow requirements are given in thousands of dollars. The company
has the following sources of funds
 * A line of credit of up to $\$100$K at an interest rate of $1\%$ per month,
 * In any one of the first three months, it can issue 90-day commercial
paper bearing a total interest of $2\%$ for the 3-month period,
 * Excess funds can be invested at an interest rate of $0.3\%$ per month.

In [ ]:
month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
cash_flow = [-150, -100, 200, -200, 50, 300]

## 2. Modeling
We begin by modeling the above short term financing problem. That is,
we write it in the language of linear programming. There are rules about
what one can and cannot do within linear programming. These rules are in
place to make certain that the remaining steps of the process (solving and
interpreting) can be successful.

Key to a linear program are the *decision variables*, *objective*, and *constraints*.

In [ ]:
!pip install gurobipy

### 2.1 Decision Variables
The decision variables represent (unknown) decisions
to be made. This is in contrast to problem data, which are values that
are either given or can be simply calculated from what is given. For the
short term financing problem, there are several possible choices of decision
variables. We will use the following decision variables: the amount $x_i$ drawn
from the line of credit in month $i$, the amount $y_i$ of commercial paper issued in month $i$, the excess funds $z_i$ in month $i$ and the company's wealth v in June. Note that, alternatively, one could use the decision variables $x_i$ and $y_i$ only, since excess funds and company's wealth can be deduced from these variables.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Short Term Financing")

# Variables. Note that the default value of the lower bound is 0
x = model.addVars(month[:-1], name="x", vtype=GRB.CONTINUOUS, ub=GRB.INFINITY)
y = model.addVars(month[:3], name="y", vtype=GRB.CONTINUOUS, ub=GRB.INFINITY)
z = model.addVars(month[:-1], name="z", vtype=GRB.CONTINUOUS, ub=GRB.INFINITY)
v = model.addVar(name="v", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)

### 2.2 Objective
Every linear program has an objective. This objective is to be either minimized or maximized. This objective has to be linear in the decision variables, which means it must be the sum of constants times decision variables.

In this case, our objective is simply to maximize $v$.

In [ ]:
# Objective
model.setObjective(v, GRB.MAXIMIZE)

### 2.3 Constraints
Every linear program also has constraints limiting feasible
decisions. Here we have three types of constraints: (i) cash inflow = cash outflow for each month, (ii) upper bounds on $x_i$, and (iii) nonnegativity of
the decision variables $x_i$, $y_i$ and $z_i$.

For example, in January ($i = 1$), there is a cash requirement of $150$.
To meet this requirement, the company can draw an amount $x_1$ from its
line of credit and issue an amount $y_1$ of commercial paper. Considering the
possibility of excess funds $z_1$ (possibly 0), the cash flow balance equation is as follows.

In [ ]:
model.addLConstr(x['Jan'] + y['Jan'] - z['Jan'] == 150, "Constraint1")

<gurobi.Constr *Awaiting Model Update*>

Next, in February ($i = 2$), there is a cash requirement of $100$. In addition,
principal plus interest of $1.01x_1$ is due on the line of credit and $1.003z_1$ is received on the invested excess funds. To meet the requirement in February, the company can draw an amount $x_2$ from its line of credit and issue an amount $y_2$ of commercial paper. So, the cash flow balance equation for February is as follows.

In [ ]:
model.addLConstr(x['Feb'] + y['Feb'] - 1.01*x['Jan'] + 1.003*z['Jan'] - z['Feb'] == 100, "Constraint2")

<gurobi.Constr *Awaiting Model Update*>

Similarly, for March we get the following equation:

In [ ]:
model.addLConstr(x['Mar'] + y['Mar'] - 1.01*x['Feb'] + 1.003*z['Feb'] - z['Mar'] == -200, "Constraint3")

<gurobi.Constr *Awaiting Model Update*>

For the months of April, May, and June, issuing a commercial paper is
no longer an option, so we will not have variables $y_4$, $y_5$, and $y_6$ in the formulation. Furthermore, any commercial paper issued between January
and March requires a payment with $2\%$ interest 3 months later. Thus, we
have the following additional equations:

In [ ]:
model.addLConstr(x['Apr'] - 1.02*y['Jan'] - 1.01*x['Mar'] + 1.003*z['Mar'] - z['Apr'] == 200, "Constraint4")
model.addLConstr(x['May'] - 1.02*y['Feb'] - 1.01*x['Apr'] + 1.003*z['Apr'] - z['May'] == -50, "Constraint5")
model.addLConstr( - 1.02*y['Mar'] - 1.01*x['May'] + 1.003*z['May'] - v == -300, "Constraint6")

<gurobi.Constr *Awaiting Model Update*>

Note that $x_i$ is the balance on the credit line in month $i$, not the incremental borrowing in month $i$. Similarly, $z_i$ represents the overall excess funds in month $i$. This choice of variables is quite convenient when it comes to writing down the upper bound and nonnegativity constraints.

In [ ]:
for i in month[:-1]:
  model.addLConstr(x[i] <= 100, "Constraint8_{}".format(i))
  # Note that the variables x,y,z are non-negative because of their definition

## 3. Solving
We now simply optimize our LP. Fixing to $0$ the paramether 'Method', force gurobi to solve by the primal simplex method. We will talk about this.






In [ ]:
model.setParam('Method', 0)
model.optimize()

Set parameter Method to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
Method  0

Optimize a model with 11 rows, 14 columns and 32 nonzeros
Model fingerprint: 0x95899dbf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 3e+02]
Presolve removed 8 rows and 3 columns
Presolve time: 0.01s
Presolved: 3 rows, 11 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.1449085e+01   0.000000e+00   7.377158e+00      0s
       2    9.2496949e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.249694915e+01


And retrive the value of the variables that attains the optimum.

In [ ]:
if model.status == GRB.OPTIMAL:
    x_values = {i: x[i].x for i in month[:-1]}
    y_values = {i: y[i].x for i in month[:3]}
    z_values = {i: z[i].x for i in month[:-1]}
    v_value = v.x

    print("Decision variables:")
    print("   x:", x_values)
    print("   y:", y_values)
    print("   z:", z_values)
    print("   v:", v_value)
    print("Objective value:", model.objVal)

Decision variables:
   x: {'Jan': 0.0, 'Feb': 0.0, 'Mar': 0.0, 'Apr': 0.0, 'May': 52.0}
   y: {'Jan': 150.0, 'Feb': 100.0, 'Mar': 151.94416749750752}
   z: {'Jan': 0.0, 'Feb': 0.0, 'Mar': 351.9441674975075, 'Apr': 0.0, 'May': 0.0}
   v: 92.49694915254233
Objective value: 92.49694915254233


### 3.1 Exercise
How would the formulation of the short-term fiancing problem above change if the commercial papers issued had a 2 month maturity instead of 3?

In [ ]:
### write the new model and optimize it
model_new = gp.Model("Short Term Financing - New")

# Variables
x = model_new.addVars(month[:-1], name="x", vtype=GRB.CONTINUOUS, ub=100)  # Line of credit (max 100K)
y = model_new.addVars(month[:4], name="y", vtype=GRB.CONTINUOUS)           # Commercial paper (issued in Jan–Apr)
z = model_new.addVars(month[:-1], name="z", vtype=GRB.CONTINUOUS)          # Investments
v = model_new.addVar(name="v", vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)  # Terminal value

# Objective: Maximize ending cash (v)
model_new.setObjective(v, GRB.MAXIMIZE)

# Balance constraints (adjusted for 2-month commercial paper maturity)
model_new.addLConstr(x['Jan'] + y['Jan'] - z['Jan'] == 150, "C1")
model_new.addLConstr(x['Feb'] + y['Feb'] - 1.01*x['Jan'] + 1.003*z['Jan'] - z['Feb'] == 100, "C2")
model_new.addLConstr(x['Mar'] + y['Mar'] - 1.01*x['Feb'] + 1.003*z['Feb'] - z['Mar'] - 1.02*y['Jan'] == -200, "C3")
model_new.addLConstr(x['Apr'] + y['Apr'] - 1.01*x['Mar'] + 1.003*z['Mar'] - z['Apr'] - 1.02*y['Feb'] == 200, "C4")
model_new.addLConstr(x['May'] - 1.01*x['Apr'] + 1.003*z['Apr'] - z['May'] - 1.02*y['Mar'] == -50, "C5")
model_new.addLConstr(-1.01*x['May'] + 1.003*z['May'] - v - 1.02*y['Apr'] == -300, "C6")
model_new.setParam('Method', 0)
model_new.optimize()
if model_new.status == GRB.OPTIMAL:
    x_values = {i: x[i].x for i in month[:-1]}
    y_values = {i: y[i].x for i in month[:4]}
    z_values = {i: z[i].x for i in month[:-1]}
    v_value = v.x

    print("Decision variables:")
    print("   x:", x_values)
    print("   y:", y_values)
    print("   z:", z_values)
    print("   v:", v_value)
    print("Objective value:", model_new.objVal)

## 4. Features of Linear Programs

Hidden in each linear programming formulation are a number of assumptions.
The usefulness of an LP model is directly related to how closely reality
matches up with these assumptions.

The first two assumptions are due to the linear form of the objective
and constraint functions. The contribution to the objective of any decision
variable is proportional to the value of the decision variable. Similarly,
the contribution of each variable to the left hand side of each constraint
is proportional to the value of the variable. This is the *Proportionality
Assumption*.

Furthermore, the contribution of a variable to the objective and constraints
is independent of the values of the other variables. This is the
*Additivity Assumption*. When additivity or proportionality assumptions are
not satisfied, a nonlinear programming model may be more appropriate.

The next assumption is the *Divisibility Assumption*: is it possible to
take any fraction of any variable? A fractional production quantity may be
worrisome if we are producing a small number of battleships or be innocuous
if we are producing millions of paperclips. If the Divisibility Assumption is
important and does not hold, then a technique called integer programming
rather than linear programming is required. This technique takes orders
of magnitude more time to find solutions but may be necessary to create
realistic solutions.

The final assumption is the *Certainty Assumption*: linear programming
allows for no uncertainty about the input parameters such as the cash-flow
requirements or interest rates we used in the short-term financing model.
Problems with uncertain parameters can be addressed using stochastic programming
or robust optimization approaches.

It is very rare that a problem will meet all of the assumptions exactly.
That does not negate the usefulness of a model. A model can still give
useful managerial insight even if reality differs slightly from the rigorous
requirements of the model.

## 5. Sensitivity Analysis
The optimal solution to a linear programming model is the most important
output of LP solvers, but it is not the only useful information they generate.
Most linear programming packages produce a tremendous amount of sensitivity
information, or information about what happens when data values
are changed.

Recall that in order to formulate a problem as a linear program, we had
to invoke a certainty assumption: we had to know what value the data took
on, and we made decisions based on that data. Often this assumption is
somewhat dubious: the data might be unknown, or guessed at, or otherwise
inaccurate. How can we determine the effect on the optimal decisions if the
values change? Clearly some numbers in the data are more important than
others. Can we find the ''important'' numbers? Can we determine the effect
of estimation errors?

Linear programming offers extensive capabilities for addressing these
questions.

In particular, we will generate a Excel-style sensitivity report, like what Solver gives with Answer Report and Sensitivity Report.

In [ ]:
import pandas as pd
# === VARIABLES SENSITIVITY REPORT ===
var_data = []
for var in model.getVars():
    var_data.append([
        var.VarName,
        var.X,
        var.RC,
        var.Obj,
        var.SAObjUp - var.Obj, # SAObjUp = smallest objective coefficient value at which the current optimal solution would remain optimal
        var.Obj - var.SAObjLow
    ])

var_df = pd.DataFrame(var_data, columns=[
    "Variable",
    "Final Value",
    "Reduced Cost",
    "Objective Coefficient",
    "Allowable Increase",
    "Allowable Decrease"
])

# === CONSTRAINTS SENSITIVITY REPORT ===
constr_data = []
for constr in model.getConstrs():
    constr_data.append([
        constr.ConstrName,
        constr.getAttr("Slack") + constr.getAttr("RHS"),
        constr.getAttr("Pi"),
        constr.getAttr("RHS"),
        constr.getAttr("SARHSUp") - constr.getAttr("RHS"), # SARHSUp = largest right-hand side value at which the current optimal solution would remain optimal
        constr.getAttr("RHS") - constr.getAttr("SARHSLow")
    ])

constr_df = pd.DataFrame(constr_data, columns=[
    "Constraint",
    "Final Value (LHS)",
    "Shadow Price",
    "RHS",
    "Allowable Increase",
    "Allowable Decrease"
])

# === PRINT OR EXPORT ===
print("\n--- VARIABLES SENSITIVITY REPORT ---")
print(var_df.round(4))

print("\n--- CONSTRAINTS SENSITIVITY REPORT ---")
print(constr_df.round(4))



--- VARIABLES SENSITIVITY REPORT ---
   Variable  Final Value  Reduced Cost  Objective Coefficient  \
0    x[Jan]       0.0000       -0.0032                    0.0   
1    x[Feb]       0.0000       -0.0000                    0.0   
2    x[Mar]       0.0000       -0.0071                    0.0   
3    x[Apr]       0.0000       -0.0032                    0.0   
4    x[May]      52.0000        0.0000                    0.0   
5    y[Jan]     150.0000        0.0000                    0.0   
6    y[Feb]     100.0000        0.0000                    0.0   
7    y[Mar]     151.9442        0.0000                    0.0   
8    z[Jan]       0.0000       -0.0040                    0.0   
9    z[Feb]       0.0000       -0.0071                    0.0   
10   z[Mar]     351.9442        0.0000                    0.0   
11   z[Apr]       0.0000       -0.0039                    0.0   
12   z[May]       0.0000       -0.0070                    0.0   
13        v      92.4969        0.0000              

The key columns for sensitivity analysis are the `Reduced Cost` (the `.RC`) and
`Shadow Price` (the attribute `Pi`) columns. The shadow price $u$ of a constraint $C$ has the following interpretation:

**If the right hand side of the constraint C changes by an amount $\Delta$, the
optimal objective value changes by $u \Delta$, as long as the amount of change
$\Delta$ is within the allowable range.**

For a linear program, the shadow price *u* is an exact figure, as long
as the amount of change $\Delta$ is within the allowable range given in the last
two columns. When the change $\Delta$ falls outside this range, the rate of change in the optimal objective value changes and the shadow price $u$ cannot be used. When this occurs, one has to resolve the linear program using the new data.


Next, we consider several examples of sensitivity questions and demonstrate
how they can be answered using shadow prices and reduced costs.

(1) Assume that Net Cash Flow in January were $-200$ (instead
of $-150$). By how much would the company's wealth decrease at the
end of June?

* The answer is in the shadow price of the January constraint, $u =1.0373$. The RHS of the January constraint would go from $150$ to
$200$, an increase of $\Delta = 50$, which is within the allowable increase. So the company's wealth in June would decrease by $1.0373 *
50,000 = \$51,865$.


(2) Assume that Net Cash Flow in March were $250$ (instead of $200$).
By how much would the company's wealth increase at the end of June?

* Again, the change $ \Delta = 50$ is within the allowable decrease,
so we can use the shadow price $u = 1.02$ to calculate the change in
objective value. The increase is $(-1.02) * (-50) = \$51,000$.


(3) Assume that the credit limit were increased from $100$ to $200$. By how
much would the company's wealth increase at the end of June?

* In each month, the change $\Delta = 100$ is within the allowable increase
($+ \infty$) and the shadow price for the credit limit constraint is $u = 0$.
So there is no effect on the company's wealth in June. Note that
non-binding constraints, such as the credit limit constraint for months
January through May, always have zero shadow price.


(4) Assume that the negative Net Cash Flow in January is partially due to the
purchase of a machine worth $ \$100,000$. The vendor allows the payment
to be made in June at an interest rate of $3\%$ for the 5-month period.
Would the company's wealth increase or decrease by using this option?
What if the interest rate for the 5-month period were $4\%$?

* The shadow price of the January constraint is $-1.0373$. This means
that reducing cash requirements in January by $\$1$ increases the wealth
in June by $\$1.0373$. In other words, the break even interest rate for
the 5-month period is $3.73\%$. So, if the vendor charges $3\%$, we should
accept, but if he charges $4\%$ we should not. Note that the analysis is
valid since the amount $\Delta = -100$ is within the allowable decrease.